In [1]:
print("Imports started..")
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
import os
import copy
import math
import random
from tqdm import tqdm

import pandas as pd
import numpy as np

# To hide OpenCV Warnings
import warnings
warnings.filterwarnings('ignore')

import cv2

from skimage.feature import hog
from skimage import data, exposure

from scipy.stats import norm
from scipy.signal import savgol_filter

import time
import torch
from torch.autograd import Variable
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models,transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader,random_split
#from __future__ import print_function
import numpy as np 
import pandas as pd
import os
import random
import matplotlib.pyplot as plt 
from torch.autograd import Function
from collections import OrderedDict
import torch.nn as nn
import math
import torchvision.models as models
import pickle
import cv2
from transformers import data
import wandb
from sklearn.model_selection import KFold
# from torchsummary import summary
from datetime import timedelta
print("Imports done.")

Imports started..


2022-11-23 23:28:25.760983: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/data/home/anagara8/.local/lib/python3.8/site-packages/cv2/../../lib64:/usr/lib/nvidia-390
2022-11-23 23:28:25.761025: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Imports done.


In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

SHAKE_PARAMETER = 50
SHAKE_PARAMETER_INDIVIDUAL = 10

ONLY_SPINE_FRAME = 224.0
DISTANCE_NORMALIZER = 150.0
ANGLE_NORMALIZER = 180.0

In [3]:
class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalEncoder, self).__init__()
        self.latent_dims = latent_dims
        self.linear1 = nn.Linear(14, 10)
        self.linear2 = nn.Linear(10, latent_dims * 2)
#         self.linear6 = nn.Linear(9, latent_dims)
#         self.linear3 = nn.Linear(16, latent_dims)

#         self.N = torch.distributions.Normal(0, 1)
#         self.N.loc = self.N.loc.cuda() # hack to get sampling on the GPU
#         self.N.scale = self.N.scale.cuda()
#         self.kl = 0

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
#         x = F.relu(self.linear2(x))
#         x = F.relu(self.linear3(x))
#         x = F.relu(self.linear4(x))
#         x = F.relu(self.linear5(x))
        x = self.linear2(x)
        mu = x[:, :self.latent_dims]
        logvar = x[:, self.latent_dims:]
#         mu =  self.linear3(x)
#         sigma = torch.exp(self.linear3(x))
#         self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.latent_dims = latent_dims
        self.linear1 = nn.Linear(self.latent_dims, 10)
        self.linear2 = nn.Linear(10, 14)
#         self.linear3 = nn.Linear(10, 11)
#         self.linear4 = nn.Linear(11, 12)
#         self.linear5 = nn.Linear(12, 13)
#         self.linear6 = nn.Linear(13, 14)

    def forward(self, z):
        z = F.relu(self.linear1(z))
#         z = F.relu(self.linear2(z))
#         z = F.relu(self.linear3(z))
#         z = F.relu(self.linear4(z))
#         z = F.relu(self.linear5(z))
        z = F.sigmoid(self.linear2(z))
#         z = self.linear3(z)
#         print(z.shape)
        return z #.reshape((-1, 1, 14))

class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar
    
    @staticmethod
    def reparameterize(mean, logvar):
        std = torch.exp(logvar / 2) # in log-space, squareroot is divide by two
        epsilon = torch.randn_like(std)
        return epsilon * std + mean

In [4]:
# !export CUDA_VISIBLE_DEVICES=0

In [5]:
model = nn.DataParallel(VariationalAutoencoder(8)).cuda()

In [6]:
model

DataParallel(
  (module): VariationalAutoencoder(
    (encoder): VariationalEncoder(
      (linear1): Linear(in_features=14, out_features=10, bias=True)
      (linear2): Linear(in_features=10, out_features=16, bias=True)
    )
    (decoder): Decoder(
      (linear1): Linear(in_features=8, out_features=10, bias=True)
      (linear2): Linear(in_features=10, out_features=14, bias=True)
    )
  )
)

In [7]:
EXPECTED_PART_LIST = ["Nose", "InnerEar", "Spine1", "Spine2", "Spine3", "Spine4", "TailStart"]
debug = False
class Node:
    '''
    Single Node in the Pose Graph.

    Parameters:
    @coordinates: x and y coordinate tuple of a part.
    @part_name: Name of the part, should be one of: 
        Nose, InnerEar, Spine1, Spine2, Spine3, Spine4, TailStart
    '''
    def __init__(self, coordinates, part_name):
        self.is_root = False
        
        self.x = coordinates[0]
        self.y = coordinates[1]

        self.edge_to = None
        self.edge_from = None
        
        self.L = None       # Distance to next node
        self.alpha = None   # Angle with next node
        
        assert part_name in EXPECTED_PART_LIST, "ERROR: " + part_name + " is not a valid part name."
        self.part = part_name

        if self.part == "TailStart":
            self.is_root = True

        if debug:
            print("DEBUG: Node " + self.part + " created! (x: " 
            + str(self.x) + ", y: " + str(self.y) + ")")
    
    def get_coordinates(self):
        return (self.x, self.y)
    
    def get_name(self):
        return part

class PoseGraph:
    '''
    Pose graph for a frame.
    
    Parameters:
    @parts: Array consisting of n parts arranged as: (x, y, part_name)
        The parts must also be in order of arrangement in the image.
    '''
    def __init__(self, parts):
        self.vertices = []
        self.edges = []
        
        self.distance_edge_to_next = []
        self.angle_edge_to_next = []

        # Node that isn't being pointed to by any other node.
        # If a node isn't pointing to anything, then it is a leaf node.
        self.root_node = None

        for i in range(len(parts)):
            # Create a Node object for the part, and add it to the
            # list of vertices
            part = Node((parts[i][0], parts[i][1]), parts[i][2])
            self.vertices.append(part)
            
            # Get the Root Node
            if part.is_root:
                self.root_node = part

            if debug:
                print("DEBUG: Added Vertex for " + part.part + " to the Pose Graph.")

            # Add a new edge if there are at least 2 vertices present
            if len(self.vertices) > 1:
                previous_part = self.vertices[-2]
                
                # Create and edge from the previous part to the current part
                previous_part.edge_to = part
                part.edge_from = previous_part
                
                self.edges.append([previous_part, part])
                
                # Add a final edge from the last part to the first part
                if len(self.vertices) == len(EXPECTED_PART_LIST):
                    part.edge_to = self.vertices[0]
                    self.vertices[0].edge_from = part
                    self.edges.append([previous_part, part])
            
                if debug:
                    print("DEBUG: Added Edge " + previous_part.part + " --> " + part.part + " to the Pose Graph.")

                    
        # Calulating the distance L and angle alpha at each part 
        # requires knowledge of the next 2 points, so it will be 
        # processed once the graph is built
        
#         def dot(vA, vB):
#             return vA[0]*vB[0]+vA[1]*vB[1]

#         def get_angle(lineA, lineB):
#             # Get nicer vector form
#             vA = [(lineA[0][0]-lineA[1][0]), (lineA[0][1]-lineA[1][1])]
#             vB = [(lineB[0][0]-lineB[1][0]), (lineB[0][1]-lineB[1][1])]
#             # Get dot prod
#             dot_prod = dot(vA, vB)
#             # Get magnitudes
#             magA = dot(vA, vA)**0.5
#             magB = dot(vB, vB)**0.5
#             # Get cosine value
#             cos_ = dot_prod/magA/magB
#             # Get angle in radians and then convert to degrees
#             angle = math.acos(dot_prod/magB/magA)
#             # Basically doing angle <- angle mod 360
#             ang_deg = math.degrees(angle)%360

#             if ang_deg-180>=0:
#                 # As in if statement
#                 return 360 - ang_deg
#             else: 

#                 return ang_deg
    
        for vertex in self.vertices:
            current_part = vertex
            previous_part = current_part.edge_from
            next_part = current_part.edge_to
            
            # Calculating Distance (L)
#             current_part.L = math.dist(next_part.get_coordinates(), current_part.get_coordinates())
            
            # Calculating the angle (alpha)
#             lineAway = [[current_part.x, current_part.y], [next_part.x, next_part.y]]
#             lineInto = [[current_part.x, current_part.y], [previous_part.x, previous_part.y]]
# #             m1 = _slope(next_part, current_part)
# #             m2 = _slope(previous_part, current_part)
# #             if m1*m2 == -1.0:
# #                 current_part.alpha = 90.0   # Angle = 90
# #             else:
# #             angle = math.atan(abs((m2 - m1) / (1 + m1 * m2)))
#             current_part.alpha = get_angle(lineAway, lineInto)
        
#         print("INFO: Create the PoseGraph with " + str(len(self.vertices)) + " vertices and " + str(len(self.edges)) + " edges.")

    def get_vertices(self):
        return self.vertices

    def get_edges(self):
        return self.edges

    def print_vertices(self):
        vertex_count = 1
        for vertex in self.vertices:
            print("Vertex " + str(vertex_count) + ": " + vertex.part 
            + "(x: " + str(vertex.x) + ", y: " + str(vertex.y) + ")")
            vertex_count += 1

    def print_edges(self):
        for edge in self.edges:
            print(edge[1].part + " --> " + edge[0].part)
            
    def get_pose_estimation(self):
        pose_estimation = []
        for vertex in self.vertices:
            pose_estimation += vertex.x, vertex.y #, vertex.L, vertex.alpha
        return pose_estimation

In [8]:
def get_poses(hdf_file, ref):
    datapoints = pd.read_hdf(hdf_file)
    dataframe_reference = ref#"DLC_resnet50_SingleVideoSpineAnnotation_video_2018Y_08M_07D_09h_51m_01s_cam_17202338-0000Oct2shuffle2_500000"

    number_of_datapoints = int(len(datapoints))
    print("#Datapoints:", number_of_datapoints)
    
    nose_x = [None]*number_of_datapoints
    nose_y = [None]*number_of_datapoints

    innerear_x = [None]*number_of_datapoints
    innerear_y = [None]*number_of_datapoints

    spine1_x = [None]*number_of_datapoints
    spine1_y = [None]*number_of_datapoints

    spine2_x = [None]*number_of_datapoints
    spine2_y = [None]*number_of_datapoints

    spine3_x = [None]*number_of_datapoints
    spine3_y = [None]*number_of_datapoints

    spine4_x = [None]*number_of_datapoints
    spine4_y = [None]*number_of_datapoints

    tailstart_x = [None]*number_of_datapoints
    tailstart_y = [None]*number_of_datapoints
    #######################################################
    previous_nose_x = 0
    previous_nose_y = 0

    previous_innerear_x = 0
    previous_innerear_y = 0

    previous_spine1_x = 0
    previous_spine1_y = 0

    previous_spine2_x = 0
    previous_spine2_y = 0

    previous_spine3_x = 0
    previous_spine3_y = 0

    previous_spine4_x = 0
    previous_spine4_y = 0

    previous_tailstart_x = 0
    previous_tailstart_y = 0
    
    # Smoothing based on previous frames:
    print("Iterating Dataframe...")
    unsure_point_count = 0
    for idx in range(0, number_of_datapoints):
        nose_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Nose']['x'])
        nose_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Nose']['y'])
        if  not abs(nose_x[idx] - previous_nose_x) > SHAKE_PARAMETER_INDIVIDUAL:
            nose_x[idx] = previous_nose_x
        if  not abs(nose_y[idx] - previous_nose_y) > SHAKE_PARAMETER_INDIVIDUAL:
            nose_y[idx] = previous_nose_y


        innerear_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['InnerEar']['x'])
        innerear_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['InnerEar']['y'])
        if  not abs(innerear_x[idx] - previous_innerear_x) > SHAKE_PARAMETER_INDIVIDUAL:
            innerear_x[idx] = previous_innerear_x
        if  not abs(innerear_y[idx] - previous_innerear_y) > SHAKE_PARAMETER_INDIVIDUAL:
            innerear_y[idx] = previous_innerear_y

        spine1_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine1']['x'])
        spine1_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine1']['y'])  
        if  not abs(spine1_x[idx] - previous_spine1_x) > SHAKE_PARAMETER_INDIVIDUAL:
            spine1_x[idx] = previous_spine1_x
        if  not abs(spine1_y[idx] - previous_spine1_y) > SHAKE_PARAMETER_INDIVIDUAL:
            spine1_y[idx] = previous_spine1_y

        spine2_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine2']['x'])
        spine2_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine2']['y'])    
        if  not abs(spine2_x[idx] - previous_spine2_x) > SHAKE_PARAMETER_INDIVIDUAL:
            spine2_x[idx] = previous_spine2_x
        if  not abs(spine2_y[idx] - previous_spine2_y) > SHAKE_PARAMETER_INDIVIDUAL:
            spine2_y[idx] = previous_spine2_y


        spine3_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine3']['x'])
        spine3_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine3']['y'])
        if  not abs(spine3_x[idx] - previous_spine3_x) > SHAKE_PARAMETER_INDIVIDUAL:
            spine3_x[idx] = previous_spine3_x
        if  not abs(spine3_y[idx] - previous_spine3_y) > SHAKE_PARAMETER_INDIVIDUAL:
            spine3_y[idx] = previous_spine3_y

        spine4_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine4']['x'])
        spine4_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine4']['y'])
        if  not abs(spine4_x[idx] - previous_spine4_x) > SHAKE_PARAMETER_INDIVIDUAL:
            spine4_x[idx] = previous_spine4_x
        if  not abs(spine4_y[idx] - previous_spine4_y) > SHAKE_PARAMETER_INDIVIDUAL:
            spine4_y[idx] = previous_spine4_y


        tailstart_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['TailStart']['x'])
        tailstart_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['TailStart']['y'])
        if  not abs(tailstart_x[idx] - previous_tailstart_x) > SHAKE_PARAMETER_INDIVIDUAL:
            tailstart_x[idx] = previous_tailstart_x
        if  not abs(tailstart_y[idx] - previous_tailstart_y) > SHAKE_PARAMETER_INDIVIDUAL:
            tailstart_y[idx] = previous_tailstart_y

    print("Done!")
    print("Smoothing Individual Points...")
    # Smoothing Individual Parts
    smooth_nose_x = [int(x) for x in savgol_filter(nose_x, 15, 3, mode='nearest')]
    smooth_nose_y = [int(y) for y in savgol_filter(nose_y, 15, 3, mode='nearest')]

    smooth_innerear_x = [int(x) for x in savgol_filter(innerear_x, 15, 3, mode='nearest')]
    smooth_innerear_y = [int(y) for y in savgol_filter(innerear_y, 15, 3, mode='nearest')]

    smooth_spine1_x = [int(x) for x in savgol_filter(spine1_x, 15, 3, mode='nearest')]
    smooth_spine1_y = [int(y) for y in savgol_filter(spine1_y, 15, 3, mode='nearest')]

    smooth_spine2_x = [int(x) for x in savgol_filter(spine2_x, 15, 3, mode='nearest')]
    smooth_spine2_y = [int(y) for y in savgol_filter(spine2_y, 15, 3, mode='nearest')]

    smooth_spine3_x = [int(x) for x in savgol_filter(spine3_x, 15, 3, mode='nearest')] 
    smooth_spine3_y = [int(y) for y in savgol_filter(spine3_y, 15, 3, mode='nearest')]

    smooth_spine4_x = [int(x) for x in savgol_filter(spine4_x, 15, 3, mode='nearest')]
    smooth_spine4_y = [int(y) for y in savgol_filter(spine4_y, 15, 3, mode='nearest')]

    smooth_tailstart_x = [int(x) for x in savgol_filter(tailstart_x, 15, 3, mode='nearest')]
    smooth_tailstart_y = [int(y) for y in savgol_filter(tailstart_y, 15, 3, mode='nearest')]
    print("Done Smoothing Points!")
    
    data_list = []
    no_append_count = 0
    previous_part_x = {"Nose":0.0, "InnerEar":0.0, "Spine1":0.0, "Spine2":0.0, "Spine3":0.0, "Spine4":0.0, "TailStart":0.0}
    previous_part_y = {"Nose":0.0, "InnerEar":0.0, "Spine1":0.0, "Spine2":0.0, "Spine3":0.0, "Spine4":0.0, "TailStart":0.0}

    print("Iterating Dataframe...")
    for idx in range(0, int(number_of_datapoints)):
        part_list = []
    #     for part in EXPECTED_PART_LIST:
    #         part_x = float(datapoints.iloc[idx][dataframe_reference][part]['x'])
    #         part_y = float(datapoints.iloc[idx][dataframe_reference][part]['y'])
    #         if  not abs(part_x - previous_part_x[part]) > SHAKE_PARAMETER_INDIVIDUAL:
    #             part_x = previous_part_x[part]
    #         else:
    #             previous_part_x[part] = part_x

    #         if  not abs(part_y - previous_part_y[part]) > SHAKE_PARAMETER_INDIVIDUAL:
    #             part_y = previous_part_y[part]
    #         else:
    #             previous_part_y[part] = part_y

        part_list.append([smooth_nose_x[idx], smooth_nose_y[idx], "Nose"])
        part_list.append([smooth_innerear_x[idx], smooth_innerear_y[idx], "InnerEar"])
        part_list.append([smooth_spine1_x[idx], smooth_spine1_y[idx], "Spine1"])
        part_list.append([smooth_spine2_x[idx], smooth_spine2_y[idx], "Spine2"])
        part_list.append([smooth_spine3_x[idx], smooth_spine3_y[idx], "Spine3"])
        part_list.append([smooth_spine4_x[idx], smooth_spine4_y[idx], "Spine4"])
        part_list.append([smooth_tailstart_x[idx], smooth_tailstart_y[idx], "TailStart"])

        try:
    #         data_list.append(PoseGraph(part_list).get_pose_estimation())
            data_list.append([smooth_nose_x[idx], smooth_nose_y[idx], 
                              smooth_innerear_x[idx], smooth_innerear_y[idx], 
                              smooth_spine1_x[idx], smooth_spine1_y[idx],
                              smooth_spine2_x[idx], smooth_spine2_y[idx],
                              smooth_spine3_x[idx], smooth_spine3_y[idx],
                              smooth_spine4_x[idx], smooth_spine4_y[idx],
                              smooth_tailstart_x[idx], smooth_tailstart_y[idx]
                             ])
        except:
            no_append_count += 1
    #         print("Didn't append for index", idx)

    print("Didn't append for a total of "+str(no_append_count)+" rows.")
    print("Done!\n")
    return data_list

In [9]:
datapoints = pd.read_hdf("/media/data_cifs/anagara8/FC-A_17202338-Akash-2022-10-02/videos/video_2018Y_08M_07D_09h_51m_01s_cam_17202338-0000DLC_resnet50_FC-A_17202338Oct2shuffle2_500000.h5")
# datapoints = pd.read_hdf("/cifs/data/tserre/CLPS_Serre_Lab/anagara8/FC-A_17202338-Akash-2022-10-02/videos/video_2018Y_08M_07D_09h_51m_01s_cam_17202338-0000DLC_resnet50_SingleVideoSpineAnnotation_video_2018Y_08M_07D_09h_51m_01s_cam_17202338-0000Oct2shuffle2_500000.h5")
# dataframe_reference = "DLC_resnet50_SingleVideoSpineAnnotation_video_2018Y_08M_07D_09h_51m_01s_cam_17202338-0000Oct2shuffle2_500000"
dataframe_reference = "DLC_resnet50_FC-A_17202338Oct2shuffle2_500000"

number_of_datapoints = int(len(datapoints))
print("#Datapoints:", number_of_datapoints)

#Datapoints: 108000


In [10]:
nose_x = [None]*number_of_datapoints
nose_y = [None]*number_of_datapoints

innerear_x = [None]*number_of_datapoints
innerear_y = [None]*number_of_datapoints

spine1_x = [None]*number_of_datapoints
spine1_y = [None]*number_of_datapoints

spine2_x = [None]*number_of_datapoints
spine2_y = [None]*number_of_datapoints

spine3_x = [None]*number_of_datapoints
spine3_y = [None]*number_of_datapoints

spine4_x = [None]*number_of_datapoints
spine4_y = [None]*number_of_datapoints

tailstart_x = [None]*number_of_datapoints
tailstart_y = [None]*number_of_datapoints
#######################################################
previous_nose_x = 0
previous_nose_y = 0

previous_innerear_x = 0
previous_innerear_y = 0

previous_spine1_x = 0
previous_spine1_y = 0

previous_spine2_x = 0
previous_spine2_y = 0

previous_spine3_x = 0
previous_spine3_y = 0

previous_spine4_x = 0
previous_spine4_y = 0

previous_tailstart_x = 0
previous_tailstart_y = 0

In [11]:
# Smoothing based on previous frames:
print("Iterating Dataframe...")
unsure_point_count = 0
for idx in range(0, number_of_datapoints):
    nose_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Nose']['x'])
    nose_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Nose']['y'])
    if  not abs(nose_x[idx] - previous_nose_x) > SHAKE_PARAMETER_INDIVIDUAL:
        nose_x[idx] = previous_nose_x
    if  not abs(nose_y[idx] - previous_nose_y) > SHAKE_PARAMETER_INDIVIDUAL:
        nose_y[idx] = previous_nose_y
    
    
    innerear_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['InnerEar']['x'])
    innerear_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['InnerEar']['y'])
    if  not abs(innerear_x[idx] - previous_innerear_x) > SHAKE_PARAMETER_INDIVIDUAL:
        innerear_x[idx] = previous_innerear_x
    if  not abs(innerear_y[idx] - previous_innerear_y) > SHAKE_PARAMETER_INDIVIDUAL:
        innerear_y[idx] = previous_innerear_y

    spine1_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine1']['x'])
    spine1_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine1']['y'])  
    if  not abs(spine1_x[idx] - previous_spine1_x) > SHAKE_PARAMETER_INDIVIDUAL:
        spine1_x[idx] = previous_spine1_x
    if  not abs(spine1_y[idx] - previous_spine1_y) > SHAKE_PARAMETER_INDIVIDUAL:
        spine1_y[idx] = previous_spine1_y

    spine2_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine2']['x'])
    spine2_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine2']['y'])    
    if  not abs(spine2_x[idx] - previous_spine2_x) > SHAKE_PARAMETER_INDIVIDUAL:
        spine2_x[idx] = previous_spine2_x
    if  not abs(spine2_y[idx] - previous_spine2_y) > SHAKE_PARAMETER_INDIVIDUAL:
        spine2_y[idx] = previous_spine2_y


    spine3_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine3']['x'])
    spine3_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine3']['y'])
    if  not abs(spine3_x[idx] - previous_spine3_x) > SHAKE_PARAMETER_INDIVIDUAL:
        spine3_x[idx] = previous_spine3_x
    if  not abs(spine3_y[idx] - previous_spine3_y) > SHAKE_PARAMETER_INDIVIDUAL:
        spine3_y[idx] = previous_spine3_y

    spine4_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine4']['x'])
    spine4_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['Spine4']['y'])
    if  not abs(spine4_x[idx] - previous_spine4_x) > SHAKE_PARAMETER_INDIVIDUAL:
        spine4_x[idx] = previous_spine4_x
    if  not abs(spine4_y[idx] - previous_spine4_y) > SHAKE_PARAMETER_INDIVIDUAL:
        spine4_y[idx] = previous_spine4_y


    tailstart_x[idx] = float(datapoints.iloc[idx][dataframe_reference]['TailStart']['x'])
    tailstart_y[idx] = float(datapoints.iloc[idx][dataframe_reference]['TailStart']['y'])
    if  not abs(tailstart_x[idx] - previous_tailstart_x) > SHAKE_PARAMETER_INDIVIDUAL:
        tailstart_x[idx] = previous_tailstart_x
    if  not abs(tailstart_y[idx] - previous_tailstart_y) > SHAKE_PARAMETER_INDIVIDUAL:
        tailstart_y[idx] = previous_tailstart_y

print("Done!")

Iterating Dataframe...
Done!


In [12]:
print("Smoothing Individual Points...")
# Smoothing Individual Parts
smooth_nose_x = [int(x) for x in savgol_filter(nose_x, 15, 3, mode='nearest')]
smooth_nose_y = [int(y) for y in savgol_filter(nose_y, 15, 3, mode='nearest')]

smooth_innerear_x = [int(x) for x in savgol_filter(innerear_x, 15, 3, mode='nearest')]
smooth_innerear_y = [int(y) for y in savgol_filter(innerear_y, 15, 3, mode='nearest')]

smooth_spine1_x = [int(x) for x in savgol_filter(spine1_x, 15, 3, mode='nearest')]
smooth_spine1_y = [int(y) for y in savgol_filter(spine1_y, 15, 3, mode='nearest')]

smooth_spine2_x = [int(x) for x in savgol_filter(spine2_x, 15, 3, mode='nearest')]
smooth_spine2_y = [int(y) for y in savgol_filter(spine2_y, 15, 3, mode='nearest')]

smooth_spine3_x = [int(x) for x in savgol_filter(spine3_x, 15, 3, mode='nearest')] 
smooth_spine3_y = [int(y) for y in savgol_filter(spine3_y, 15, 3, mode='nearest')]

smooth_spine4_x = [int(x) for x in savgol_filter(spine4_x, 15, 3, mode='nearest')]
smooth_spine4_y = [int(y) for y in savgol_filter(spine4_y, 15, 3, mode='nearest')]

smooth_tailstart_x = [int(x) for x in savgol_filter(tailstart_x, 15, 3, mode='nearest')]
smooth_tailstart_y = [int(y) for y in savgol_filter(tailstart_y, 15, 3, mode='nearest')]
print("Done Smoothing Points!")

Smoothing Individual Points...
Done Smoothing Points!


In [13]:
data_list = []
no_append_count = 0
previous_part_x = {"Nose":0.0, "InnerEar":0.0, "Spine1":0.0, "Spine2":0.0, "Spine3":0.0, "Spine4":0.0, "TailStart":0.0}
previous_part_y = {"Nose":0.0, "InnerEar":0.0, "Spine1":0.0, "Spine2":0.0, "Spine3":0.0, "Spine4":0.0, "TailStart":0.0}

print("Iterating Dataframe...")
for idx in range(0, int(number_of_datapoints)):
    part_list = []
#     for part in EXPECTED_PART_LIST:
#         part_x = float(datapoints.iloc[idx][dataframe_reference][part]['x'])
#         part_y = float(datapoints.iloc[idx][dataframe_reference][part]['y'])
#         if  not abs(part_x - previous_part_x[part]) > SHAKE_PARAMETER_INDIVIDUAL:
#             part_x = previous_part_x[part]
#         else:
#             previous_part_x[part] = part_x
            
#         if  not abs(part_y - previous_part_y[part]) > SHAKE_PARAMETER_INDIVIDUAL:
#             part_y = previous_part_y[part]
#         else:
#             previous_part_y[part] = part_y

    part_list.append([smooth_nose_x[idx], smooth_nose_y[idx], "Nose"])
    part_list.append([smooth_innerear_x[idx], smooth_innerear_y[idx], "InnerEar"])
    part_list.append([smooth_spine1_x[idx], smooth_spine1_y[idx], "Spine1"])
    part_list.append([smooth_spine2_x[idx], smooth_spine2_y[idx], "Spine2"])
    part_list.append([smooth_spine3_x[idx], smooth_spine3_y[idx], "Spine3"])
    part_list.append([smooth_spine4_x[idx], smooth_spine4_y[idx], "Spine4"])
    part_list.append([smooth_tailstart_x[idx], smooth_tailstart_y[idx], "TailStart"])
    
    try:
#         data_list.append(PoseGraph(part_list).get_pose_estimation())
        data_list.append([smooth_nose_x[idx], smooth_nose_y[idx], 
                          smooth_innerear_x[idx], smooth_innerear_y[idx], 
                          smooth_spine1_x[idx], smooth_spine1_y[idx],
                          smooth_spine2_x[idx], smooth_spine2_y[idx],
                          smooth_spine3_x[idx], smooth_spine3_y[idx],
                          smooth_spine4_x[idx], smooth_spine4_y[idx],
                          smooth_tailstart_x[idx], smooth_tailstart_y[idx]
                         ])
    except:
        no_append_count += 1
#         print("Didn't append for index", idx)

print("Didn't append for a total of "+str(no_append_count)+" rows.")
print("Done!\n")

Iterating Dataframe...
Didn't append for a total of 0 rows.
Done!



In [14]:
# Index x = 640 -> 0
# Index y = 480 -> 1

In [15]:
class Pose_Dataset(torch.utils.data.Dataset):
    def __init__(self, pose_list, augmentations = None):
        super(Pose_Dataset, self).__init__()
        self.pose_list = pose_list
        self.augmentations = augmentations
    
    def __len__(self):
        return len(self.pose_list)

    def __getitem__(self, idx):
        # Creating a copy to avoid changing the values
        current_pose = copy.copy(self.pose_list[idx])
#         print("CP:", current_pose)
        # Removing Location information by centering
#         spine1_x = pose[8]
#         spine1_y = pose[9]
#         spine2_x = pose[12]
#         spine2_y = pose[13]
        only_spine_normalization_x = (current_pose[6]+current_pose[4])/2 - (ONLY_SPINE_FRAME/2)
        only_spine_normalization_y = (current_pose[7]+current_pose[5])/2 - (ONLY_SPINE_FRAME/2)

#         print("OSN", only_spine_normalization_x, only_spine_normalization_y)
# #         Normalizing Pose:
#         for i in range(28):
#             if i % 4 == 0:
#                 current_pose[i] = (current_pose[i] - only_spine_normalization_x)/ONLY_SPINE_FRAME
#             elif (i-1) % 4 == 0:
#                 current_pose[i] = (current_pose[i] - only_spine_normalization_y)/ONLY_SPINE_FRAME
#             elif (i-2) % 4 == 0:
#                 current_pose[i] = current_pose[i]/DISTANCE_NORMALIZER
#             else:
#                 current_pose[i] = current_pose[i]/ANGLE_NORMALIZER
#         print(current_pose)

#         Normalizing Pose
        for i in range(14):
#             print(i, current_pose[i], (current_pose[i] - only_spine_normalization_x)/ONLY_SPINE_FRAME)
            if i % 2 == 0:
                current_pose[i] = (current_pose[i] - only_spine_normalization_x)/ONLY_SPINE_FRAME
            else:
                current_pose[i] = (current_pose[i] - only_spine_normalization_y)/ONLY_SPINE_FRAME
#             print(i, current_pose[i])
#         for i in range(14):
#             if i % 2 == 0:
#                 current_pose[i] = (current_pose[i]/640.0)
#             else:
#                 current_pose[i] = (current_pose[i]/480.0)
            
        return torch.tensor(current_pose)#, only_spine_normalization_x, only_spine_normalization_y

In [16]:
data_list_data = data_list
m=len(data_list_data)

print("Length:",m, "-> Training set size:", int(math.ceil(m-m*0.2)), "| Validation set size:", int(m*0.2))
train_data, val_data = data_list_data[:int(math.ceil(m-m*0.2))], data_list_data[int(math.ceil(m-m*0.2)):]

Length: 108000 -> Training set size: 86400 | Validation set size: 21600


In [17]:
batch_size=4096
# train_set_loader = torch.utils.data.DataLoader(Pose_Dataset(data_list), batch_size=batch_size, num_workers = 4, shuffle=True)
train_set_loader = torch.utils.data.DataLoader(Pose_Dataset(train_data), batch_size=batch_size, num_workers = 4, shuffle=False)
valid_set_loader = torch.utils.data.DataLoader(Pose_Dataset(val_data), batch_size=batch_size, num_workers = 4, shuffle=False)
print("Converted to DataLoader")

Converted to DataLoader


In [18]:
train_set_loader.dataset[5]

tensor([0.2522, 0.5737, 0.3728, 0.5201, 0.4576, 0.5067, 0.5424, 0.4933, 0.6138,
        0.5246, 0.6808, 0.5960, 0.7344, 0.6942])

In [19]:
# np.save("pose_training_set.npy", train_set_loader)
# np.save("pose_valid_set.npy", valid_set_loader)
train_set_loader = np.load("pose_training_set.npy", allow_pickle=True).item()
valid_set_loader = np.load("pose_valid_set.npy", allow_pickle=True).item()
m=len(train_set_loader) + len(valid_set_loader)

In [20]:
number_of_epochs = 1000
learningRate = 0.01

In [21]:
autoencoder_optimizer = optim.Adam(model.parameters(), lr = learningRate, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(autoencoder_optimizer, step_size=100, gamma = 0.01)

In [22]:
list_train_loss = []
list_val_loss = []
list_train_MSE = []
list_val_MSE = []
list_train_KLD = []
list_val_KLD = []

train_set_size = int(math.ceil(m-m*0.2))
val_set_size = int(m*0.2)

In [23]:
# Gradient Flow
def plot_grad_flow(named_parameters):
    ave_grads = []
    layers = []
    for n, p in named_parameters:
#         p = p.detach().cpu()
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean().cpu().numpy())
#     plt.figure(figsize=(15, 15))
    plt.plot(ave_grads, alpha=0.3, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(xmin=0, xmax=len(ave_grads))
#     plt.ylim(ymin=0, ymax=5)
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)

In [28]:
def loss_functionVAE(recon_x, x, mu, logvar):
    KLD = torch.mean(-0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(),dim=1),dim=0)
    MSE = nn.MSELoss()(recon_x, x)
    beta = 0
    return MSE + beta*KLD, MSE, beta*KLD, KLD
# number_of_epochs = 200

In [30]:
for epoch in range(number_of_epochs):
    start_time = time.time()

    train_run_loss = 0 
    train_run_KLD = 0 
    train_run_MSE = 0 

    val_run_loss = 0
    val_run_KLD = 0
    val_run_MSE = 0
    model.train(True) # For training
    for image_batch1 in train_set_loader:
        image_batch1 = image_batch1.cuda()
        autoencoder_optimizer.zero_grad()
        enc_dec_img, mu, logvar = model(image_batch1)
        original_train_loss, train_MSE, train_KLD, raw_train_KLD = loss_functionVAE(enc_dec_img, image_batch1, mu, logvar)
        train_loss = original_train_loss.data.item()
        # Backward pass
        original_train_loss.backward()
#             plot_grad_flow(model.named_parameters())
        autoencoder_optimizer.step()
#             scheduler.step()
        del(original_train_loss)
        train_run_loss += train_loss

    model.eval()
    for image_batch2 in valid_set_loader:
        image_batch2 = image_batch2.cuda()
        autoencoder_optimizer.zero_grad()
        enc_dec_img2, mu, logvar = model(image_batch2)
        original_val_loss, val_MSE, val_KLD, raw_val_KLD = loss_functionVAE(enc_dec_img2, image_batch2, mu, logvar)
        val_loss = original_val_loss.data.item()
        val_run_loss += val_loss
        del(original_val_loss)
#             val_run_MSE += val_MSE/val_set_size
#             val_run_KLD += val_KLD
        
        
    print('[%d] Loss -> Training: %.7f | Validation: %.7f (Real KLD:%.7f) in %.2f seconds.' % (epoch + 1, train_run_loss, val_run_loss, raw_val_KLD, time.time() - start_time))
    list_val_loss.append(val_run_loss)
    list_train_loss.append(train_run_loss)
#     list_val_MSE.append(val_run_MSE)
#     list_train_MSE.append(train_run_MSE)
#     list_val_KLD.append(val_run_KLD)
#     list_train_KLD.append(train_run_KLD)
    val_run_loss = 0.0
    train_run_loss = 0.0
    
print('Finished Training')

[1] Loss -> Training: 0.0006995 | Validation: 0.0002474 (Real KLD:26.6054287) in 2.60 seconds.
[2] Loss -> Training: 0.0007021 | Validation: 0.0002231 (Real KLD:26.6066666) in 2.40 seconds.
[3] Loss -> Training: 0.0006926 | Validation: 0.0002518 (Real KLD:26.6107922) in 2.25 seconds.
[4] Loss -> Training: 0.0007306 | Validation: 0.0002351 (Real KLD:26.5889874) in 2.38 seconds.
[5] Loss -> Training: 0.0006784 | Validation: 0.0002444 (Real KLD:26.5476456) in 2.29 seconds.
[6] Loss -> Training: 0.0007404 | Validation: 0.0002483 (Real KLD:26.5306358) in 2.53 seconds.
[7] Loss -> Training: 0.0007069 | Validation: 0.0002248 (Real KLD:26.4887695) in 2.46 seconds.
[8] Loss -> Training: 0.0007187 | Validation: 0.0002622 (Real KLD:26.4753475) in 2.61 seconds.
[9] Loss -> Training: 0.0007512 | Validation: 0.0002376 (Real KLD:26.4419880) in 2.60 seconds.
[10] Loss -> Training: 0.0006970 | Validation: 0.0002610 (Real KLD:26.4784031) in 2.70 seconds.
[11] Loss -> Training: 0.0007725 | Validation: 0.

[87] Loss -> Training: 0.0006592 | Validation: 0.0002264 (Real KLD:25.9835529) in 2.31 seconds.
[88] Loss -> Training: 0.0006488 | Validation: 0.0002251 (Real KLD:26.0574131) in 2.28 seconds.
[89] Loss -> Training: 0.0006481 | Validation: 0.0002238 (Real KLD:26.0854130) in 2.33 seconds.
[90] Loss -> Training: 0.0006440 | Validation: 0.0002260 (Real KLD:26.0702839) in 2.33 seconds.
[91] Loss -> Training: 0.0006492 | Validation: 0.0002300 (Real KLD:26.1018410) in 2.45 seconds.
[92] Loss -> Training: 0.0006527 | Validation: 0.0002234 (Real KLD:26.1519909) in 2.26 seconds.
[93] Loss -> Training: 0.0006440 | Validation: 0.0002235 (Real KLD:26.2130852) in 2.28 seconds.
[94] Loss -> Training: 0.0006450 | Validation: 0.0002219 (Real KLD:26.2487926) in 2.25 seconds.
[95] Loss -> Training: 0.0006407 | Validation: 0.0002253 (Real KLD:26.2689381) in 2.37 seconds.
[96] Loss -> Training: 0.0006482 | Validation: 0.0002283 (Real KLD:26.2763805) in 2.32 seconds.
[97] Loss -> Training: 0.0006522 | Valid

[172] Loss -> Training: 0.0007015 | Validation: 0.0002284 (Real KLD:25.6496143) in 2.33 seconds.
[173] Loss -> Training: 0.0006722 | Validation: 0.0002353 (Real KLD:25.6058044) in 2.28 seconds.
[174] Loss -> Training: 0.0006931 | Validation: 0.0002338 (Real KLD:25.7012959) in 2.34 seconds.
[175] Loss -> Training: 0.0006819 | Validation: 0.0002343 (Real KLD:25.8265991) in 2.35 seconds.
[176] Loss -> Training: 0.0006752 | Validation: 0.0002403 (Real KLD:25.8278522) in 2.30 seconds.
[177] Loss -> Training: 0.0006882 | Validation: 0.0002313 (Real KLD:25.7833385) in 2.47 seconds.
[178] Loss -> Training: 0.0006746 | Validation: 0.0002321 (Real KLD:25.7861462) in 2.39 seconds.
[179] Loss -> Training: 0.0006915 | Validation: 0.0002368 (Real KLD:25.8185177) in 3.01 seconds.
[180] Loss -> Training: 0.0006912 | Validation: 0.0002372 (Real KLD:25.8233910) in 2.34 seconds.
[181] Loss -> Training: 0.0006807 | Validation: 0.0002428 (Real KLD:25.8247051) in 2.26 seconds.
[182] Loss -> Training: 0.0007

[257] Loss -> Training: 0.0007111 | Validation: 0.0002404 (Real KLD:25.7646179) in 2.87 seconds.
[258] Loss -> Training: 0.0007099 | Validation: 0.0002484 (Real KLD:25.7226295) in 2.83 seconds.
[259] Loss -> Training: 0.0007279 | Validation: 0.0002416 (Real KLD:25.7161102) in 3.28 seconds.
[260] Loss -> Training: 0.0007094 | Validation: 0.0002477 (Real KLD:25.7158394) in 3.45 seconds.
[261] Loss -> Training: 0.0007406 | Validation: 0.0002484 (Real KLD:25.7112274) in 4.64 seconds.
[262] Loss -> Training: 0.0007319 | Validation: 0.0002540 (Real KLD:25.7333107) in 3.10 seconds.
[263] Loss -> Training: 0.0007179 | Validation: 0.0002472 (Real KLD:25.6798153) in 2.81 seconds.
[264] Loss -> Training: 0.0007263 | Validation: 0.0002566 (Real KLD:25.5778580) in 2.94 seconds.
[265] Loss -> Training: 0.0007322 | Validation: 0.0002463 (Real KLD:25.5713940) in 3.32 seconds.
[266] Loss -> Training: 0.0007366 | Validation: 0.0002474 (Real KLD:25.6593685) in 3.46 seconds.
[267] Loss -> Training: 0.0007

[342] Loss -> Training: 0.0006560 | Validation: 0.0002259 (Real KLD:25.9203625) in 3.48 seconds.
[343] Loss -> Training: 0.0006639 | Validation: 0.0002297 (Real KLD:25.9079800) in 3.27 seconds.
[344] Loss -> Training: 0.0006570 | Validation: 0.0002265 (Real KLD:25.9308624) in 3.51 seconds.
[345] Loss -> Training: 0.0006684 | Validation: 0.0002430 (Real KLD:25.9602375) in 3.44 seconds.
[346] Loss -> Training: 0.0006873 | Validation: 0.0002357 (Real KLD:25.9013710) in 3.47 seconds.
[347] Loss -> Training: 0.0006802 | Validation: 0.0002279 (Real KLD:25.9079056) in 3.29 seconds.
[348] Loss -> Training: 0.0007243 | Validation: 0.0002583 (Real KLD:25.9535999) in 3.52 seconds.
[349] Loss -> Training: 0.0007266 | Validation: 0.0002362 (Real KLD:25.9564476) in 3.45 seconds.
[350] Loss -> Training: 0.0007101 | Validation: 0.0002528 (Real KLD:25.9155903) in 4.04 seconds.
[351] Loss -> Training: 0.0007920 | Validation: 0.0002943 (Real KLD:25.8500004) in 3.44 seconds.
[352] Loss -> Training: 0.0007

...

[597] Loss -> Training: 0.0007226 | Validation: 0.0002394 (Real KLD:25.8761406) in 2.28 seconds.
[598] Loss -> Training: 0.0007030 | Validation: 0.0002461 (Real KLD:25.8259602) in 2.28 seconds.
[599] Loss -> Training: 0.0007401 | Validation: 0.0002521 (Real KLD:25.8533077) in 2.42 seconds.
[600] Loss -> Training: 0.0007385 | Validation: 0.0002379 (Real KLD:25.8959179) in 2.35 seconds.
[601] Loss -> Training: 0.0007111 | Validation: 0.0002605 (Real KLD:25.8290615) in 2.42 seconds.
[602] Loss -> Training: 0.0007658 | Validation: 0.0002744 (Real KLD:25.6285419) in 2.52 seconds.
[603] Loss -> Training: 0.0007919 | Validation: 0.0002597 (Real KLD:25.4416924) in 2.33 seconds.
[604] Loss -> Training: 0.0007698 | Validation: 0.0002723 (Real KLD:25.4781895) in 2.33 seconds.
[605] Loss -> Training: 0.0008408 | Validation: 0.0002758 (Real KLD:25.5735207) in 2.65 seconds.
[606] Loss -> Training: 0.0008406 | Validation: 0.0002724 (Real KLD:25.4743919) in 2.32 seconds.
[607] Loss -> Training: 0.0007

[682] Loss -> Training: 0.0007292 | Validation: 0.0002385 (Real KLD:25.4682579) in 2.49 seconds.
[683] Loss -> Training: 0.0006945 | Validation: 0.0002444 (Real KLD:25.4761868) in 2.41 seconds.
[684] Loss -> Training: 0.0007119 | Validation: 0.0002536 (Real KLD:25.4964638) in 2.38 seconds.
[685] Loss -> Training: 0.0007183 | Validation: 0.0002406 (Real KLD:25.5093060) in 2.48 seconds.
[686] Loss -> Training: 0.0006936 | Validation: 0.0002345 (Real KLD:25.4694653) in 2.48 seconds.
[687] Loss -> Training: 0.0007026 | Validation: 0.0002401 (Real KLD:25.3778706) in 2.33 seconds.
[688] Loss -> Training: 0.0006895 | Validation: 0.0002368 (Real KLD:25.4159317) in 2.33 seconds.
[689] Loss -> Training: 0.0006847 | Validation: 0.0002405 (Real KLD:25.4955559) in 2.58 seconds.
[690] Loss -> Training: 0.0006913 | Validation: 0.0002410 (Real KLD:25.5136166) in 2.49 seconds.
[691] Loss -> Training: 0.0006769 | Validation: 0.0002274 (Real KLD:25.4347916) in 2.56 seconds.
[692] Loss -> Training: 0.0006

[767] Loss -> Training: 0.0006955 | Validation: 0.0002442 (Real KLD:25.4231339) in 2.30 seconds.
[768] Loss -> Training: 0.0007129 | Validation: 0.0002430 (Real KLD:25.4643059) in 2.33 seconds.
[769] Loss -> Training: 0.0006968 | Validation: 0.0002344 (Real KLD:25.5425606) in 2.34 seconds.
[770] Loss -> Training: 0.0007108 | Validation: 0.0002500 (Real KLD:25.5650139) in 2.35 seconds.
[771] Loss -> Training: 0.0007185 | Validation: 0.0002509 (Real KLD:25.4801903) in 2.49 seconds.
[772] Loss -> Training: 0.0007213 | Validation: 0.0002443 (Real KLD:25.4052334) in 2.34 seconds.
[773] Loss -> Training: 0.0007199 | Validation: 0.0002451 (Real KLD:25.3780727) in 2.35 seconds.
[774] Loss -> Training: 0.0007239 | Validation: 0.0002477 (Real KLD:25.3994560) in 2.30 seconds.
[775] Loss -> Training: 0.0007205 | Validation: 0.0002338 (Real KLD:25.4297161) in 2.30 seconds.
[776] Loss -> Training: 0.0006994 | Validation: 0.0002578 (Real KLD:25.4341526) in 2.36 seconds.
[777] Loss -> Training: 0.0007

[852] Loss -> Training: 0.0007151 | Validation: 0.0002561 (Real KLD:24.8432865) in 2.37 seconds.
[853] Loss -> Training: 0.0007192 | Validation: 0.0002277 (Real KLD:24.8696041) in 2.30 seconds.
[854] Loss -> Training: 0.0006722 | Validation: 0.0002531 (Real KLD:24.9793701) in 2.40 seconds.
[855] Loss -> Training: 0.0007124 | Validation: 0.0002318 (Real KLD:25.1396217) in 2.45 seconds.
[856] Loss -> Training: 0.0006739 | Validation: 0.0002335 (Real KLD:25.2164783) in 2.42 seconds.
[857] Loss -> Training: 0.0006834 | Validation: 0.0002348 (Real KLD:25.2119770) in 2.36 seconds.
[858] Loss -> Training: 0.0006661 | Validation: 0.0002252 (Real KLD:25.2315388) in 2.46 seconds.
[859] Loss -> Training: 0.0006558 | Validation: 0.0002370 (Real KLD:25.2788086) in 2.40 seconds.
[860] Loss -> Training: 0.0006741 | Validation: 0.0002257 (Real KLD:25.3696995) in 2.57 seconds.
[861] Loss -> Training: 0.0006565 | Validation: 0.0002292 (Real KLD:25.4951134) in 2.41 seconds.
[862] Loss -> Training: 0.0006

[937] Loss -> Training: 0.0007822 | Validation: 0.0002360 (Real KLD:25.0140686) in 2.31 seconds.
[938] Loss -> Training: 0.0007389 | Validation: 0.0002966 (Real KLD:25.0247974) in 2.46 seconds.
[939] Loss -> Training: 0.0007877 | Validation: 0.0002375 (Real KLD:24.8723793) in 2.51 seconds.
[940] Loss -> Training: 0.0007006 | Validation: 0.0002613 (Real KLD:24.8597565) in 2.42 seconds.
[941] Loss -> Training: 0.0007674 | Validation: 0.0002387 (Real KLD:24.9594097) in 2.35 seconds.
[942] Loss -> Training: 0.0006991 | Validation: 0.0002428 (Real KLD:24.9894619) in 2.47 seconds.
[943] Loss -> Training: 0.0007031 | Validation: 0.0002439 (Real KLD:24.9443569) in 2.41 seconds.
[944] Loss -> Training: 0.0006935 | Validation: 0.0002412 (Real KLD:24.9136925) in 2.44 seconds.
[945] Loss -> Training: 0.0006856 | Validation: 0.0002299 (Real KLD:25.0502663) in 2.64 seconds.
[946] Loss -> Training: 0.0006762 | Validation: 0.0002376 (Real KLD:25.1768227) in 2.33 seconds.
[947] Loss -> Training: 0.0006

In [26]:
# old_train = []
# old_val = []

In [27]:
old_train += list_train_loss
old_val += list_val_loss

NameError: name 'old_train' is not defined

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(len(old_train)),old_train,'tab:orange',label='Training Loss')
plt.plot(range(len(old_val)),old_val,'tab:blue',label='Validation Loss')
plt.title("Loss Curve")
plt.xlabel("Epochs")
# plt.ylim([0.00000, 0.001])
plt.ylabel("Loss")
plt.legend()

In [ ]:
def pose_decoder(pose_input):
    
    # Create a copy to not change original list
    pose = copy.copy(pose_input)
    
#     osn_x = pose[1]
#     osn_y = pose[2]
#     pose = pose[0]
    
#     print("OSN_x:",osn_x)
#     print("OSN_y:",osn_y)
    
    if not type(pose) == type([]) and not type(pose) == type(np.array([])):
        pose = pose.detach().cpu().numpy()
    print("poSE:", pose)
        
#     print("POSE:", pose)
    
#     print("Before:", pose[0], pose[1])
    
    # Check if normalization is needed:
#     if pose[0] < 1.0:
#         print("Normalizing!")
#         # Normalization
# #         for i in range(28):
# #             if i % 4 == 0:
# #                 pose[i] = pose[i]*224.0 + osn_x
# #             elif (i-1) % 4 == 0:
# #                 pose[i] = pose[i]*224.0 + osn_y
# #             elif (i-2) % 4 == 0:
# #                 pose[i] = pose[i]*DISTANCE_NORMALIZER
# #             else:
# #                 pose[i] = pose[i]*ANGLE_NORMALIZER
#         for i in range(14):
#             if i % 2 == 0:
#                 pose[i] = pose[i]*ONLY_SPINE_FRAME + osn_x
#             else:
#                 pose[i] = pose[i]*ONLY_SPINE_FRAME + osn_y
            
#     print("After:", pose[0], pose[1])
    
#     nose_x = pose[0]
#     nose_y = pose[1]
#     nose_L = pose[2]
#     nose_alpha = pose[3]
#     innerear_x = pose[4]
#     innerear_y = pose[5]
#     innerear_L = pose[6]
#     innerear_alpha = pose[7]
#     spine1_x = pose[8]
#     spine1_y = pose[9]
#     spine1_L = pose[10]
#     spine1_alpha = pose[11]
#     spine2_x = pose[12]
#     spine2_y = pose[13]
#     spine2_L = pose[14]
#     spine2_alpha = pose[15]
#     spine3_x = pose[16]
#     spine3_y = pose[17]
#     spine3_L = pose[18]
#     spine3_alpha = pose[19]
#     spine4_x = pose[20]
#     spine4_y = pose[21]
#     spine4_L = pose[22]
#     spine4_alpha = pose[23]
#     tailstart_x = pose[24]
#     tailstart_y = pose[25]
#     tailstart_L = pose[26]
#     tailstart_alpha = pose[27]
    
    nose_x = pose[0]*ONLY_SPINE_FRAME
    nose_y = pose[1]*ONLY_SPINE_FRAME
    innerear_x = pose[2]*ONLY_SPINE_FRAME
    innerear_y = pose[3]*ONLY_SPINE_FRAME
    spine1_x = pose[4]*ONLY_SPINE_FRAME
    spine1_y = pose[5]*ONLY_SPINE_FRAME
    spine2_x = pose[6]*ONLY_SPINE_FRAME
    spine2_y = pose[7]*ONLY_SPINE_FRAME
    spine3_x = pose[8]*ONLY_SPINE_FRAME
    spine3_y = pose[9]*ONLY_SPINE_FRAME
    spine4_x = pose[10]*ONLY_SPINE_FRAME
    spine4_y = pose[11]*ONLY_SPINE_FRAME
    tailstart_x = pose[12]*ONLY_SPINE_FRAME
    tailstart_y = pose[13]*ONLY_SPINE_FRAME
    
    # Create Empty Image:
    empty_image = np.zeros([224,224,3], dtype=np.uint8)
    empty_image.fill(255)
    
    only_spine_normalization_x = (spine2_x+spine1_x)/2 - 112
    only_spine_normalization_y = (spine2_y+spine1_y)/2 - 112
    
    print("OSN_x2:", only_spine_normalization_x)
    print("OSN_y2:", only_spine_normalization_y)
    
    only_spine_nose_x = int(nose_x - only_spine_normalization_x)
    only_spine_nose_y = int(nose_y - only_spine_normalization_y)
    only_spine_innerear_x = int(innerear_x - only_spine_normalization_x)
    only_spine_innerear_y = int(innerear_y - only_spine_normalization_y)
    only_spine_spine1_x = int(spine1_x - only_spine_normalization_x)
    only_spine_spine1_y = int(spine1_y - only_spine_normalization_y)
    only_spine_spine2_x = int(spine2_x - only_spine_normalization_x)
    only_spine_spine2_y = int(spine2_y - only_spine_normalization_y)
    only_spine_spine3_x = int(spine3_x - only_spine_normalization_x)
    only_spine_spine3_y = int(spine3_y - only_spine_normalization_y)
    only_spine_spine4_x = int(spine4_x - only_spine_normalization_x)
    only_spine_spine4_y = int(spine4_y - only_spine_normalization_y)
    only_spine_tailstart_x = int(tailstart_x - only_spine_normalization_x)
    only_spine_tailstart_y = int(tailstart_y - only_spine_normalization_y)

    # Draw the Spine
    pointsInside = [(only_spine_nose_x, only_spine_nose_y),
                    (only_spine_innerear_x, only_spine_innerear_y),
                    (only_spine_spine1_x, only_spine_spine1_y),
                    (only_spine_spine2_x, only_spine_spine2_y),
                    (only_spine_spine3_x, only_spine_spine3_y),
                    (only_spine_spine4_x, only_spine_spine4_y),
                    (only_spine_tailstart_x, only_spine_tailstart_y)]
    print(pointsInside)
    
#     print(pointsInside)

    for index, item in enumerate(pointsInside): 
        if index == len(pointsInside) - 1:
            break
        empty_image = cv2.line(empty_image, item, pointsInside[index + 1], [0, 0, 0], 1)

    # Add the points
    empty_image = cv2.circle(empty_image, (only_spine_nose_x, only_spine_nose_y), 4, (109,27,190), -1)
    empty_image = cv2.circle(empty_image, (only_spine_innerear_x, only_spine_innerear_y), 4, (65,186,184), -1)
    empty_image = cv2.circle(empty_image, (only_spine_spine1_x, only_spine_spine1_y), 4, (125,215,162), -1)
    empty_image = cv2.circle(empty_image, (only_spine_spine2_x, only_spine_spine2_y), 4, (161,206,141), -1)
    empty_image = cv2.circle(empty_image, (only_spine_spine3_x, only_spine_spine3_y), 4, (181,187,125), -1)
    empty_image = cv2.circle(empty_image, (only_spine_spine4_x, only_spine_spine4_y), 4, (218,130,93), -1)
    empty_image = cv2.circle(empty_image, (only_spine_tailstart_x, only_spine_tailstart_y), 4, (198,19,32), -1)

    plt.figure(figsize=(10,10))
    plt.title("Mouse Pose")
#     plt.xticks([])
#     plt.yticks([])
#     empty_image = cv2.flip(empty_image, 1)
    plt.imshow(empty_image)

In [ ]:
pose_decoder(train_set_loader.dataset[0])

In [ ]:
model.eval()
op = model(torch.unsqueeze(torch.tensor(train_set_loader.dataset[0]), 0).cuda())
print("Target:", train_set_loader.dataset[0])
print("Output:", op[0][0].detach().cpu().numpy())
pose_decoder(op[0][0])

In [ ]:
def pose_encoder(pose_input):
    
    # Create a copy to not change original list
    pose = copy.copy(pose_input)
    
    if not type(pose) == type([]):
        pose = pose.detach().cpu().numpy()
    
    # Normalization
    for i in range(28):
        if i % 4 == 0:
            pose[i] = pose[i]/640.0
        elif (i-1) % 4 == 0:
            pose[i] = pose[i]/480.0
        elif (i-2) % 4 == 0:
            pose[i] = pose[i]/150.0
        else:
            pose[i] = pose[i]/90.0
    return pose

# Inference

In [ ]:
pd.read_hdf("/cifs/data/tserre/CLPS_Serre_Lab/anagara8/AnalysisVideos/17202338_a/Pre/video_2018Y_08M_12D_03h_56m_34s_cam_17202338-0000DLC_resnet50_FC-A_17202338Oct2shuffle2_500000.h5").iloc[0]

In [ ]:
expt_pre_hour2 = get_poses("/cifs/data/tserre/CLPS_Serre_Lab/anagara8/AnalysisVideos/17202338_a/Pre/video_2018Y_08M_12D_03h_56m_34s_cam_17202338-0000DLC_resnet50_FC-A_17202338Oct2shuffle2_500000.h5", "DLC_resnet50_FC-A_17202338Oct2shuffle2_500000")

In [ ]:
expt_pre_hour3 = get_poses("/cifs/data/tserre/CLPS_Serre_Lab/anagara8/AnalysisVideos/17202338_a/Pre/video_2018Y_08M_12D_04h_56m_34s_cam_17202338-0000DLC_resnet50_FC-A_17202338Oct2shuffle2_500000.h5", "DLC_resnet50_FC-A_17202338Oct2shuffle2_500000")

In [ ]:
expt_post_hour2 = get_poses("/cifs/data/tserre/CLPS_Serre_Lab/anagara8/AnalysisVideos/17202338_a/Post/Day1/video_2018Y_08M_14D_03h_21m_05s_cam_17202338-0000DLC_resnet50_FC-A_17202338Oct2shuffle2_500000.h5", "DLC_resnet50_FC-A_17202338Oct2shuffle2_500000")

In [ ]:
expt_post_hour3 = get_poses("/cifs/data/tserre/CLPS_Serre_Lab/anagara8/AnalysisVideos/17202338_a/Post/Day1/video_2018Y_08M_14D_04h_21m_05s_cam_17202338-0000DLC_resnet50_FC-A_17202338Oct2shuffle2_500000.h5", "DLC_resnet50_FC-A_17202338Oct2shuffle2_500000")

In [ ]:
vame= get_poses("/cifs/data/tserre/CLPS_Serre_Lab/anagara8/FC-A_17202338-Akash-2022-10-02/videos/video_2018Y_08M_07D_09h_51m_01s_cam_17202338-0000DLC_resnet50_SingleVideoSpineAnnotation_video_2018Y_08M_07D_09h_51m_01s_cam_17202338-0000Oct2shuffle2_500000.h5", dataframe_reference)

In [ ]:
vame_latents = get_latents_from_pose(vame)

In [ ]:
def quick_posification(pose):
    current_pose = pose
    only_spine_normalization_x = (current_pose[6]+current_pose[4])/2 - (ONLY_SPINE_FRAME/2)
    only_spine_normalization_y = (current_pose[7]+current_pose[5])/2 - (ONLY_SPINE_FRAME/2)
    for i in range(14):
        if i % 2 == 0:
            current_pose[i] = (current_pose[i] - only_spine_normalization_x)/ONLY_SPINE_FRAME
        else:
            current_pose[i] = (current_pose[i] - only_spine_normalization_y)/ONLY_SPINE_FRAME

    return torch.tensor(current_pose)

In [ ]:
def get_latents_from_pose(poses):
    model.eval()
    all_pose_latents = []
    for i in range(len(poses)):
        if i%5000 == 0:
            print("Processing pose", str(i)+"...")
        tensor_pose = quick_posification(poses[i]).unsqueeze(0).cuda()
        with torch.no_grad():
            current_latent = model.module.encoder(tensor_pose)[0]
        all_pose_latents.append(np.array(current_latent.cpu()))
    return all_pose_latents

In [ ]:
def create_df(latents):
#     s = 97800
    n = len(latents)
    latent_dataframe = pd.DataFrame(latents)
    counts = [i for i in range(n)]
    counts_df = pd.DataFrame(counts)

    latent_dataframe.insert(8, "Time", counts, True)
    
    return latent_dataframe

In [ ]:
vame_latent_dataframe = create_df(vame_latents2)
vame_latent_dataframe

In [ ]:
len(expt_post_latents2)

In [ ]:
def plot_latents(latent_dataframe, n_frames = None):
    
    if n_frames == None:
        n_frames = len(latent_dataframe)
        
    dataframe = latent_dataframe[:n_frames]
    colours = ["#DC143C", "#FF0000", "#FF7F50", "#FF6347", 
           "#FF8C00", "#FFD700", "#B8860B", "#DAA520", 
           "#EEE8AA", "#FFFF00", "#9ACD32", "#556B2F", 
           "#7CFC00", "#006400", "#32CD32", "#8FBC8F",
           "#2E8B57", "#66CDAA", "#3CB371", "#20B2AA",
           "#2F4F4F", "#00FFFF", "#00CED1", "#7FFFD4",
           "#5F9EA0", "#FF1493", "#6495ED", "#00BFFF",
           "#1E90FF", "#000080", "#8A2BE2", "#8B008B"]

    random.seed(42)
    random.shuffle(colours)

    for i in range(8):
        plt.plot(dataframe["Time"], list(map(lambda x:x-((i+1)*3.5), dataframe[i])), color=colours[i], label='Latent '+str(i))
    # Plotting batches of latents:
    #     if (i+1)%4 == 0:
    #         print(i)
    #         plt.rcParams["figure.figsize"] = (20,8)
    #         plt.legend(loc='upper right')
    #         plt.show()
    plt.title("Latents across "+str(n_frames)+" frames", fontsize=18)
    plt.xticks([])
    plt.yticks([])
    plt.rcParams["figure.figsize"] = (20,10)
    plt.legend(loc='upper right',prop={'size': 16.735})
    plt.show()

In [ ]:
plot_latents(vame_latent_dataframe)